<!-- ## 환경설정 -->

In [ ]:
from dotenv import load_dotenv
load_dotenv()

: 

In [171]:
# !uv add langchain langchain-openai langchain-community pypdf langchain-postgres langchain_huggingface sentence-transformers

In [ ]:
import os
from langchain_openai import ChatOpenAI`
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tempfile

<!-- ### PDF 로드 함수 및 데이터 추출 -->

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
import camelot

pdf_path = "./bin_pdf_data/BIN_업무협조_요청_VAN_20250801.pdf"

loader = PyPDFLoader(pdf_path)
pages = loader.load() 

# 텍스트 로드
total_document = "\n".join([page.page_content for page in pages])

# 표 
# 2) 표 로드 (텍스트 기반 PDF에서만)
tables = camelot.read_pdf(pdf_path, pages="all", flavor="lattice")  # or 'stream'
print(len(tables))
dfs = [t.df for t in tables]  # 각 표를 pandas DataFrame으로

dfs

1


[     0          1           2       3     4      5      6     7     8     9   \
 0    순번        BIN  자체카드\n구  분     발급사  기관코드    브랜드   회원구분  카드구분  카드등급  전건대행   
 1  2069    9407-57          자체  헥토파이낸셜    57  LOCAL     개인    체크   플러스  대행불가   
 2  2070    9200-57          자체  헥토파이낸셜    57  LOCAL  개인+기업    체크   플러스  대행불가   
 3  2071  5302-0035          자체  헥토파이낸셜    57     MC     개인    체크   플러스  대행불가   
 4  2072    9483-51          자체    비씨카드    50  LOCAL     개인    신용    PT  대행가능   
 5  2073    9483-52          자체    비씨카드    50  LOCAL     기업    신용    PT  대행가능   
 6  2074    5465-17          자체    비씨카드    50     MC     개인    신용    PT  대행가능   
 7  2075    5483-64          자체    비씨카드    50     MC     기업    신용    PT  대행가능   
 8  2076    5485-27          자체    수협은행    07     MC     개인    신용    PT  대행가능   
 
      10    11        12  
 0  신용대행  건별대행  할부가능\n여부  
 1  거래중계  대행불가      할부불가  
 2  거래중계  대행불가      할부불가  
 3  거래중계  대행불가      할부불가  
 4  대행가능  대행불가      할부가능  
 5  대행가능  대행불가      할부불가  

In [174]:
total_document

'정산팀-40-00083\n순번 BIN 자체카드\n구  분 발급사 기관코드 브랜드 회원구분 카드구분 카드등급 전건대행 신용대행 건별대행 할부가능\n여부\n2069 9407-57 자체 헥토파이낸셜 57 LOCAL 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2070 9200-57 자체 헥토파이낸셜 57 LOCAL 개인+기업 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2071 5302-0035 자체 헥토파이낸셜 57 MC 개인 체크 플러스 대행불가 거래중계 대행불가 할부불가\n2072 9483-51 자체 비씨카드 50 LOCAL 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n2073 9483-52 자체 비씨카드 50 LOCAL 기업 신용 PT 대행가능 대행가능 대행불가 할부불가\n2074 5465-17 자체 비씨카드 50 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n2075 5483-64 자체 비씨카드 50 MC 기업 신용 PT 대행가능 대행가능 대행불가 할부불가\n2076 5485-27 자체 수협은행 07 MC 개인 신용 PT 대행가능 대행가능 대행불가 할부가능\n * 신규고객사 "헥토파이낸셜" 매출표 표기 : 헥토카드\nㅇ 일정에 따른 BIN번호 전산반영 및 회신\nㅇ BIN자체관리 가맹점으로 BIN번호 재통지 및 반영여부 확인 요망\n  참      조 : 신용카드 담당 부서장\n비씨카드주식회사\n서울시 중구 을지로 170 을지트윈타워\n담당자 : 정산팀 이성희 대리, T: 02)520-8373, E: ssung@bccard.com\n  문서번호 : 2025-08-01\n  수      신 : 수신처 참조\n  제      목 : [비씨카드] BIN 관련 업무협조 요청\n1.\xa0귀 사(원)의 무궁한 발전을 기원합니다.\n2.\xa0당사 신규 BIN(Bank Identification Number) 생성에 따라 다음과 같이 요청하오니 협조하여 주시기 바랍니다.\nㅇ 할부대행은 6개월 이내에서 가능합니다.\n3. 내  

In [175]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal
from typing import List, Optional

# PDF 표 구조 
class BinRow(BaseModel):
    bin: str  = Field(description="BIN번호(8자리 또는 6자리)")                                                      # BIN번호 "8자리(또는 6자리) BIN 문자열"
    issuer: str = Field(description="발급사")                                                                     # 발급사
    instCd: str = Field(description="기관코드")                                                                   # 기관코드
    brand: str  = Field(description="브랜드사")                                                                   # 브랜드사
    memberType: Literal["개인","법인","개인+기업"] = Field(description="회원구분")                                   # 회원구분
    cardType:   Literal["체크", "신용", "선불"] = Field(description="카드구분")                                     # 카드구분
    cardRank:   Literal["플러스", "PT", "골드", "체크", "일반", "우량"] = Field(description="대행한도등급")           # 대행한도등급
    agentFull:  Literal["Y","N"]  = Field(description="전건대행")                                                 # 전건대행
    agentCredit: Literal["Y","N"] = Field(description="신용대행")                                                 # 신용대행
    agentPerTxn: Literal["Y","N"] = Field(description="건별대행")                                                 # 건별대행    
    installmentYN: Literal["Y","N"] = Field(description="할부 가능여부")                                           # 할부 가능여부

class ExtractionResult(BaseModel):
    rows: List[BinRow]  = []
    notes: Optional[str]


In [176]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser


#parser = StructuredOutputParser.from_typed_schema(ExtractionResult)
parser = PydanticOutputParser(pydantic_object=ExtractionResult)
# print(parser.get_format_instructions())
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"BinRow": {"properties": {"bin": {"description": "BIN번호(8자리 또는 6자리)", "title": "Bin", "type": "string"}, "issuer": {"description": "발급사", "title": "Issuer", "type": "string"}, "instCd": {"description": "기관코드", "title": "Instcd", "type": "string"}, "brand": {"description": "브랜드사", "title": "Brand", "type": "string"}, "memberType": {"description": "회원구분", "enum": ["개인", "법인", "개인+기업"], "title": "Membertype", "type": "string"}, "cardType": {"description": "카드구분", "enum": ["체크", "신용", "선불"], "title": "Cardtype", "type": "string"}, "card

In [177]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.output_parsers import StructuredOutputParser

model = ChatOpenAI(model="gpt-4.1-mini")


system = """
당신은 카드 BIN 데이터 추출 보조자입니다.
다음 규칙으로만 JSON을 생성하세요:
- JSON 스키마를 엄격히 준수
- 불확실 시 '?' 사용
- 근거 문맥을 source_text에 200자 이내로 첨부
- 페이지 번호를 page_ref에 기록
"""

human = """PDF에서 추출한 BIN번호 관련 dataframe이 아래에 제공됩니다.
BIN번호, 발급사, 기관코드, 브랜드, 회원구분, 카드구분, 카드등급, 완전대행, 신용대행, 건별대행, 할부가능여부 모두 표준화해 rows 배열로 반환하세요.
텍스트:
{joined_text}

표(있다면 CSV로 직렬화됨):
{tables_csv}

반환은 반드시 유효한 JSON 한 덩어리만:
{format_instructions}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

# 텍스트 페이지 합치기 (너무 길면 샘플링/요약 후 단계적 호출)
joined_text = "\n\n".join([f"[p{idx+1}] {d.page_content}" for idx, d in enumerate(pages)])

def df_to_csv(df):
    import io
    return df.to_csv(index=False)

tables_csv = "\n\n".join([df_to_csv(df) for df in dfs]) if len(dfs) else "N/A"

chain = prompt | model | parser
result = chain.invoke({
    "joined_text": joined_text[:100000],  # 토큰 초과 방지
    "tables_csv": tables_csv[:50000],
    "format_instructions": parser.get_format_instructions()
})

In [178]:
result.rows

[BinRow(bin='940757', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='920057', issuer='헥토파이낸셜', instCd='57', brand='LOCAL', memberType='개인+기업', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='53020035', issuer='헥토파이낸셜', instCd='57', brand='MC', memberType='개인', cardType='체크', cardRank='플러스', agentFull='N', agentCredit='N', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='948351', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='개인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='Y'),
 BinRow(bin='948352', issuer='비씨카드', instCd='50', brand='LOCAL', memberType='법인', cardType='신용', cardRank='PT', agentFull='Y', agentCredit='Y', agentPerTxn='N', installmentYN='N'),
 BinRow(bin='546517', issuer='비씨카드', instCd='50', brand='MC', memberType='개인', cardT

In [179]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import Engine, Result

engine_url = os.getenv("ORACLE_ENGINE_URL")
engine: Engine = create_engine(engine_url, pool_pre_ping=True)  # 필요 시 echo=True

stmt = text(
    """
    SELECT card_bin, iss_inst_cd, stip_lmt_lvl_cd, psn_corp_card_cl, card_brand_cd, card_typ_cd, mbr_cmpy_cd, ist_psbl_yn, stip_psbl_yn, stip_ist_psbl_yn, wgt_cl
    FROM INF_CARD_BIN
    WHERE 1=1
    AND CARD_BIN = :bin
    """
)
params = {
    "bin": "920057"
}

with engine.connect() as conn:
    select_result: Result = conn.execute(stmt, params)
    rows = [dict(r._mapping) for r in select_result.fetchall()]

print(len(rows))
print(rows)

1
[{'card_bin': '920057', 'iss_inst_cd': '0100', 'stip_lmt_lvl_cd': '00', 'psn_corp_card_cl': '1', 'card_brand_cd': 'L', 'card_typ_cd': '1', 'mbr_cmpy_cd': '057', 'ist_psbl_yn': 'N', 'stip_psbl_yn': 'N', 'stip_ist_psbl_yn': 'N', 'wgt_cl': 'N'}]


In [180]:
# pip install -U "langchain>=0.2.24" "langchain-openai>=0.2.2" "pydantic>=2.5"
# from typing import List, Optional, Literal, Dict, Any
# from pydantic import BaseModel, Field

# # 비교 결과의 "필드별" 리포트
# class FieldResult(BaseModel):
#     ok: bool = Field(..., description="이 필드가 규칙에 따라 일치하는지")
#     db: Any = Field(..., description="DB 측 비교값(정규화 후)")
#     pdf: Any = Field(..., description="PDF 측 비교값(정규화 후)")
#     note: Optional[str] = Field(None, description="비고/설명(맵핑근거, 보정 규칙 등)")

# class ValidationReport(BaseModel):
#     all_passed: bool = Field(..., description="모든 규칙 통과 여부")
#     results: Dict[str, FieldResult] = Field(..., description="필드별 비교 결과")
#     mismatches: List[str] = Field(default_factory=list, description="불일치 항목 key 목록")

In [181]:
from typing import List, Optional, Union, Dict
from pydantic import BaseModel, Field

# 얕은 JSON 타입 (재귀 제거: 안정적인 스키마 생성)
JsonScalar = Union[str, int, float, bool, None]
JsonFlat   = Union[
    JsonScalar,
    Dict[str, JsonScalar],
    List[JsonScalar],
    List[Dict[str, JsonScalar]],
]

class FieldResult(BaseModel):
    ok: bool = Field(..., description="규칙 일치 여부")
    db: JsonFlat = Field(..., description="DB 측 비교값(정규화 후)")
    pdf: JsonFlat = Field(..., description="PDF 측 비교값(정규화 후)")
    note: Optional[str] = Field(None, description="설명/근거")

# ✅ Dict 대신 리스트로(맵 제약 회피)
class ResultEntry(BaseModel):
    key: str = Field(..., description="필드 키 (예: 'card_bin↔bin')")
    result: FieldResult

class ValidationReport(BaseModel):
    # 전 필드 required (기본값/Optional 제거)
    all_passed: bool = Field(..., description="전체 통과 여부")
    results: List[ResultEntry] = Field(..., description="필드별 결과 리스트")
    mismatches: List[str] = Field(..., description="불일치 키 목록")

In [197]:

# SYSTEM_RULES = """
# 너는 카드 BIN 데이터 검증기다. 아래 '검증 규칙'을 반드시 적용하여 DB↔PDF가 동일한지 판단하고,
# 요청된 스키마(ValidationReport)로만 JSON을 출력하라.

# [검증 규칙: (왼쪽=DB, 오른쪽=PDF)]
# 1) card_bin : bin → 문자 동일 비교
# 2) iss_inst_cd : issuer
#    매핑표: 0201=농협, 0500=신한, 0202=수협, 0203=씨티, 0301=제주, 0302=광주, 0303=전북, 0100=비씨
#    PDF의 issuer가 {{농협,신한,수협,씨티,제주,광주,전북,비씨(또는 BC)}} 중 하나가 아니면 '비씨'로 간주하여 비교
# 3) stip_lmt_lvl_cd : cardRank
#    21=법인, 13=PT, 17=골드, 16=우량, 15=일반, 00=플러스
#    단, PDF memberType이 '법인'이면 DB 코드는 반드시 21 이어야 한다
# 4) psn_corp_card_cl : memberType
#    1=개인일반, 2=법인카드
#    단, PDF가 '개인+기업'이면 DB 코드는 1 이어야 한다
# 5) card_brand_cd : brand
#    A=AMEX, J=JCB, L=LOCAL, V=VISA, M=MASTER, D=DINERS, C=은련, G=GLOBAL, K=JUST TOUCH, N=CONA
#    PDF brand는 대소문자 구분 없이 비교
# 6) card_typ_cd : cardType
#    0=신용, 1=체크, 2=선불
# 7) mbr_cmpy_cd : instCd
#    숫자 비교(앞의 0 제거 후 정수 비교)
# 8) ist_psbl_yn : installmentYN (Y/N 동일 비교)
# 9) stip_psbl_yn : (agentFull, agentCredit)
#    agentFull='Y' AND agentCredit='Y' → 'Y', 그 외 'N'
# 10) stip_ist_psbl_yn : (agentFull, agentCredit, installmentYN)
#    agentFull='Y' AND agentCredit='Y' AND installmentYN='Y' → 'Y', 그 외 'N'
# 11) wgt_cl : agentPerTxn → 동일 비교
# """

SYSTEM_RULES="""
너는 카드 BIN 데이터 검증기다.
아래 "검증 규칙"에 따라 DB 값 ↔ PDF 값을 비교하라.
출력은 반드시 ValidationReport 스키마(JSON)로만 생성한다.

[검증 규칙]

1. card_bin ↔ bin
   - 단순 문자열 동일성 비교.

2. iss_inst_cd ↔ issuer
   - 매핑표: {{0201: 농협, 0500: 신한, 0202: 수협, 0203: 씨티,
             0301: 제주, 0302: 광주, 0303: 전북, 0100: 비씨}}
   - PDF issuer가 {{농협, 신한, 수협, 씨티, 제주, 광주, 전북, 비씨, BC}} 외면
     "비씨"로 간주하여 비교.

3. stip_lmt_lvl_cd ↔ cardRank
   - 매핑표: {{21: 법인, 13: PT, 17: 골드, 16: 우량, 15: 일반, 00: 플러스}}
   - 단, PDF memberType이 "법인"이면 DB 값은 반드시 21이어야 함.

4. psn_corp_card_cl ↔ memberType
   - 매핑표: {{1: 개인일반, 2: 법인카드}}
   - 단, PDF 값이 "개인+기업"이면 DB 값은 1이어야 함.

5. card_brand_cd ↔ brand
   - 매핑표: {{A: AMEX, J: JCB, L: LOCAL, V: VISA, M: MASTER,
             D: DINERS, C: 은련, G: GLOBAL, K: JUST TOUCH, N: CONA}}
   - PDF brand는 대소문자 무시.

6. card_typ_cd ↔ cardType
   - 매핑표: {{0: 신용, 1: 체크, 2: 선불}}
   - "체크카드"도 "체크"로 처리.

7. mbr_cmpy_cd ↔ instCd
   - 앞자리 0 제거 후 정수 비교.

8. ist_psbl_yn ↔ installmentYN
   - Y/N 직접 비교.

9. stip_psbl_yn ↔ agentFull & agentCredit
   - 둘 다 'Y' → 'Y', 아니면 'N'.

10. stip_ist_psbl_yn ↔ agentFull & agentCredit & installmentYN
    - 모두 'Y' → 'Y', 아니면 'N'.

11. wgt_cl ↔ agentPerTxn
    - Y/N 동일 비교.

"""

HUMAN = """
아래는 PDF에서 추출한 BinRow 1건과 DB 레코드 1건이다.
규칙을 적용하여 정규화/매핑한 비교값으로 필드별 결과를 만들고, ValidationReport 스키마(JSON) 한 덩어리만 출력하라.

[PDF BinRow]
{pdf_json}

[DB Row]
{db_json}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_RULES.strip()),
    ("human", HUMAN.strip())
])

In [193]:
import json

structured_llm = model.with_structured_output(ValidationReport)  # ✅ 권장 경로
chain = prompt | structured_llm  # 프롬프트 → LLM(스키마 바인딩)

pdf_row = result.rows[1]
db_row  = rows[0]



In [194]:
db_row

{'card_bin': '920057',
 'iss_inst_cd': '0100',
 'stip_lmt_lvl_cd': '00',
 'psn_corp_card_cl': '1',
 'card_brand_cd': 'L',
 'card_typ_cd': '1',
 'mbr_cmpy_cd': '057',
 'ist_psbl_yn': 'N',
 'stip_psbl_yn': 'N',
 'stip_ist_psbl_yn': 'N',
 'wgt_cl': 'N'}

In [195]:
pdf_row.model_dump()

{'bin': '920057',
 'issuer': '헥토파이낸셜',
 'instCd': '57',
 'brand': 'LOCAL',
 'memberType': '개인+기업',
 'cardType': '체크',
 'cardRank': '플러스',
 'agentFull': 'N',
 'agentCredit': 'N',
 'agentPerTxn': 'N',
 'installmentYN': 'N'}

In [196]:
report: ValidationReport = chain.invoke({
    "pdf_json": json.dumps(pdf_row.model_dump(), ensure_ascii=False, indent=2),
    "db_json": json.dumps(db_row, ensure_ascii=False, indent=2),
})

print(report.all_passed)
for entry in report.results:                  # report.results: List[ResultEntry]
    k = entry.key
    v = entry.result                          # v: FieldResult
    print(k, "=>", v.ok, "| DB:", v.db, "| PDF:", v.pdf, "|", v.note or "")

True
card_bin↔bin => True | DB: 920057 | PDF: 920057 | 
iss_inst_cd↔issuer => True | DB: 0100 | PDF: 비씨 | PDF issuer '헥토파이낸셜'은 매핑표 외부, '비씨'로 간주
stip_lmt_lvl_cd↔cardRank => True | DB: 00 | PDF: 00 | PDF cardRank '플러스' 매핑
psn_corp_card_cl↔memberType => True | DB: 1 | PDF: 1 | PDF memberType '개인+기업'은 DB 1이어야 함
card_brand_cd↔brand => True | DB: L | PDF: L | 대소문자 무시 매핑
card_typ_cd↔cardType => True | DB: 1 | PDF: 1 | PDF '체크'는 매핑상 1
mbr_cmpy_cd↔instCd => True | DB: 057 | PDF: 57 | 앞자리 0 제거 후 정수 비교
ist_psbl_yn↔installmentYN => True | DB: N | PDF: N | 
stip_psbl_yn↔agentFull & agentCredit => True | DB: N | PDF: N | 'N' & 'N' 이므로 'N'
stip_ist_psbl_yn↔agentFull & agentCredit & installmentYN => True | DB: N | PDF: N | 셋 다 'N'이므로 'N'
wgt_cl↔agentPerTxn => True | DB: N | PDF: N | 


In [5]:
prompt_template = """
당신은 금융 전문가입니다. 다음 문서를 읽고 사용자의 질문에 대한 답변을 제공해주세요.

<document>
{document}
</document>

<question>
{question}
</question>

** 답변 ** :
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()

result = chain.invoke({"document": total_document, "question": "VIP 콜센터 전화번호?"})

print(result)

VIP 콜센터 전화번호는 1566-7890입니다.


<!-- # 텍스트 분할(Text Splitting) 

- 대규모 텍스트 문서를 처리할 때 매우 중요한 전처리 단계
- 고려사항:
    1. 문서의 구조와 형식
    2. 원하는 청크 크기
    3. 문맥 보존의 중요도
    4. 처리 속도 


# Document Loader
[https://python.langchain.com/api_reference/community/document_loaders.html#]

1. CharacterTextSplitter
- 분할 기준: 지정된 단일 문자(예: \n, 공백 등)를 기준으로 텍스트를 분할합니다.​
- 특징:
단순한 문자 기반 분할 방식으로, 구조화되지 않은 텍스트에 적합합니다.​
문맥이나 의미를 고려하지 않기 때문에, 문장이나 단어가 중간에 잘릴 수 있습니다.

2. RecursiveCharacterTextSplitter
- 분할 기준: 여러 구분자(기본값: ["\n\n", "\n", " ", ""])를 우선순위에 따라 재귀적으로 적용하여 텍스트를 분할합니다.​
- 특징:
문단 → 문장 → 단어 순으로 분할을 시도하여, 텍스트의 의미와 문맥을 최대한 보존합니다.​
구조화된 텍스트나 자연어 처리에서 의미 단위를 유지하려는 경우에 적합합니다.

3. TokenTextSplitter
- 분할 기준: 토큰 수를 기준으로 텍스트를 분할합니다.​
- 특징: LLM의 토큰 제한을 고려하여 텍스트를 분할하므로, 모델 입력에 최적화된 형태로 텍스트를 준비할 수 있습니다.​ 언어별 토크나이저를 활용하여 정확한 토큰 단위 분할이 가능합니다 -->


In [ ]:
from langchain_text_splitters import CharacterTextSplitter 

# 텍스트 분할기 초기화 (기본 설정값 적용 )
text_splitter = CharacterTextSplitter(

    # CharacterTextSplitter의 기본 설정값
    separator = "\n\n",         # 청크 구분자: 두 개의 개행문자
    is_separator_regex = False,  # 구분자가 정규식인지 여부

    # TextSplitter의 기본 설정값
    chunk_size = 500,          # 청크 길이
    chunk_overlap = 100,        # 청크 중첩
    length_function = len,      # 길이 함수 (문자열 길이)
    keep_separator = False,     # 구분자 유지 여부
    add_start_index = False,   # 시작 인덱스 추가 여부
    strip_whitespace = True,   # 공백 제거 여부
)

# 텍스트 분할 - split_text() 메서드 사용
texts = text_splitter.split_text(total_document)

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(texts)}')

# 첫 번째 분할된 텍스트 출력
print(f'첫 번째 분할된 텍스트: {texts[0]}')

print(f'--------------------------------')

In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,             # 청크 크기
    chunk_overlap=100,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pages)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("-" * 100)
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 85
각 청크의 길이: [20, 488, 491, 490, 463, 469, 205, 497, 474, 459, 459, 499, 315, 483, 473, 493, 477, 485, 482, 455, 477, 470, 343, 497, 465, 442, 458, 477, 229, 469, 493, 452, 487, 493, 460, 324, 481, 474, 491, 484, 476, 488, 364, 458, 490, 481, 495, 376, 500, 477, 460, 452, 474, 474, 492, 496, 239, 499, 482, 468, 311, 485, 472, 191, 489, 491, 493, 497, 491, 454, 479, 239, 497, 484, 495, 333, 475, 463, 490, 326, 491, 450, 497, 134, 177]

BC 플래티늄 카드 상품서비스 가이드
----------------------------------------------------------------------------------------------------
BC 플래티늄 카드 상품서비스 가이드

01
BC PLATINUM 
SERVICE
당신의 가치를 만드는 카드 
BC 플래티늄카드
회원님과 함께 신용사회의 새 지평을 열어온 비씨카드가 오랫동안 다져온 신용카드 
서비스 노하우를 바탕으로 회원님을 VIP 회원으로 모시게 되었습니다. 
BC 플래티늄카드는 기존의 서비스와는 차별화되는 프리미엄급 서비스만을 제시
하여 회원님을 차원이 다른 고품격의 세계로
----------------------------------------------------------------------------------------------------
 국내선 항공탑승시 동반자  
항공권 무료 제공 서비스, 전세계 1,000여개 공항의 귀빈급 전용라운지 이용  
서비스, 회원님의 사용내역을 바로 문자메세지로 통지하는 바로알림(SM